In [166]:
import sqlite3
import requests
import pandas as pd
conn = sqlite3.connect('database.sqlite')

In [2]:
c = conn.cursor()

In [11]:
c.execute('''
SELECT * 
FROM Matches
WHERE HomeTeam = 'Bayern Munich'
LIMIT 1;''').fetchall()

[(22, 'D1', 2009, '2009-11-22', 'Bayern Munich', 'Leverkusen', 1, 1, 'D')]

In [17]:
c.execute('''
SELECT COUNT(DISTINCT(AwayTeam)) 
FROM Matches;''').fetchall()

[(128,)]

In [160]:
c.execute('''
SELECT * 
FROM Matches
LIMIT 100;''').fetchall()

[(1, 'D2', 2009, '2010-04-04', 'Oberhausen', 'Kaiserslautern', 2, 1, 'H'),
 (2, 'D2', 2009, '2009-11-01', 'Munich 1860', 'Kaiserslautern', 0, 1, 'A'),
 (3, 'D2', 2009, '2009-10-04', 'Frankfurt FSV', 'Kaiserslautern', 1, 1, 'D'),
 (4, 'D2', 2009, '2010-02-21', 'Frankfurt FSV', 'Karlsruhe', 2, 1, 'H'),
 (5, 'D2', 2009, '2009-12-06', 'Ahlen', 'Karlsruhe', 1, 3, 'A'),
 (6, 'D2', 2009, '2010-04-03', 'Union Berlin', 'Karlsruhe', 1, 1, 'D'),
 (7, 'D2', 2009, '2009-08-14', 'Paderborn', 'Karlsruhe', 2, 0, 'H'),
 (8, 'D2', 2009, '2010-03-08', 'Bielefeld', 'Karlsruhe', 0, 1, 'A'),
 (9, 'D2', 2009, '2009-09-26', 'Kaiserslautern', 'Karlsruhe', 2, 0, 'H'),
 (10, 'D2', 2009, '2009-11-21', 'Hansa Rostock', 'Karlsruhe', 2, 1, 'H'),
 (11, 'D2', 2009, '2009-12-19', 'Greuther Furth', 'Karlsruhe', 1, 4, 'A'),
 (12, 'D2', 2009, '2010-04-16', 'Koblenz', 'Karlsruhe', 2, 2, 'D'),
 (13, 'D2', 2009, '2010-03-21', 'Oberhausen', 'Karlsruhe', 1, 0, 'H'),
 (14, 'D2', 2009, '2009-09-14', 'Cottbus', 'Karlsruhe', 2, 4,

In [33]:
c.execute('''
SELECT * 
FROM Matches
WHERE Season = 2016 OR Season = 2017 AND Div != 'D2'
LIMIT 1;''').fetchall()

[(4152, 'D1', 2016, '2017-04-04', 'Hoffenheim', 'Bayern Munich', 1, 0, 'H')]

In [51]:
#sum of home goals
c.execute('''
CREATE TABLE home_goals AS
SELECT HomeTeam, SUM(FTHG), Div, Season 
FROM Matches
WHERE Season = 2011 AND Div != 'D2'
GROUP BY(HomeTeam);''').fetchall()

[]

In [52]:
#sum of away goals
c.execute('''
CREATE TABLE away_goals AS
SELECT AwayTeam, SUM(FTAG), Div, Season 
FROM Matches
WHERE Season = 2011 AND Div != 'D2'
GROUP BY(AwayTeam);''').fetchall()

[]

In [130]:
#this joins away and home goal tables to sum 2011 season goals
c.execute(''' 
CREATE TABLE total_goals_2011 AS
SELECT h.HomeTeam AS TeamName, h.Div, h.Season, a.'SUM(FTAG)' + h.'SUM(FTHG)' AS Total_Goals
FROM away_goals a
JOIN home_goals h ON (a.AwayTeam = h.HomeTeam);''').fetchall()

[]

In [119]:
# 2011 Home Wins

c.execute('''
CREATE TABLE home_win_2011 AS
SELECT TeamName, goals_2011.Season, goals_2011.Div, SUM(CASE WHEN (goals_2011.TeamName = Matches.HomeTeam AND Matches.FTR = 'H')
            THEN 1 
            ELSE 0
            END) AS HomeWins
    FROM Matches
    JOIN goals_2011 ON (goals_2011.TeamName = Matches.HomeTeam)
    WHERE Matches.Season = 2011 AND Matches.Div != 'D2'
    GROUP BY goals_2011.TeamName;''').fetchall()

    

[]

In [120]:
c.execute('''
CREATE TABLE away_win_2011 AS
SELECT TeamName, goals_2011.Season, goals_2011.Div, SUM(CASE WHEN (goals_2011.TeamName = Matches.AwayTeam AND Matches.FTR = 'A')
            THEN 1 
            ELSE 0
            END) AS AwayWins
    FROM Matches
    JOIN goals_2011 ON (goals_2011.TeamName = Matches.AwayTeam)
    WHERE Matches.Season = 2011 AND Matches.Div != 'D2'
    GROUP BY goals_2011.TeamName;''').fetchall()

[]

In [132]:
#this joins away and home win tables to sum 2011 season goals
c.execute(''' 
CREATE TABLE Team_Stats_2011 AS
SELECT TeamName, g.Div, g.Season, Total_Goals, AwayWins+HomeWins AS Total_Wins
FROM total_goals_2011 g
JOIN home_win_2011 USING (TeamName)
JOIN away_win_2011 USING (TeamName);''').fetchall()

[]

In [281]:
df_team_stats_2011 = pd.read_sql_query('''SELECT TeamName, g.Div, g.Season, Total_Goals, AwayWins+HomeWins AS Total_Wins
FROM total_goals_2011 g
JOIN home_win_2011 USING (TeamName)
JOIN away_win_2011 USING (TeamName);''', conn)

In [142]:
import json

def get_keys(path):
    with open(path) as f:
        return json.load(f)

keys = get_keys(".secret/darksky_api.json")

api_key = keys['api_key']

In [156]:
date = '2018-06-11'
response = requests.get('https://api.darksky.net/forecast/'+api_key+'/52.520008,13.404954,'+date+'T12:00:00?exclude=hourly,currently,flags') 
if response.status_code == 200: #200 is an HTTP status code that means "everything is ok". ITs good to check the status_code in an if statement to avoid throwing and error
    response_dict = response.json()
else:
    print("Error: unable to retrieve books.  Server responded with status code", response.status_code)


In [164]:
c.execute('''
SELECT * 
FROM Matches
WHERE Season = 2011 AND Div != 'D2'
LIMIT 5;''').fetchall()

[(1092, 'D1', 2011, '2012-03-31', 'Nurnberg', 'Bayern Munich', 0, 1, 'A'),
 (1093, 'D1', 2011, '2011-12-11', 'Stuttgart', 'Bayern Munich', 1, 2, 'A'),
 (1094, 'D1', 2011, '2011-08-13', 'Wolfsburg', 'Bayern Munich', 0, 1, 'A'),
 (1095, 'D1', 2011, '2011-11-27', 'Mainz', 'Bayern Munich', 3, 2, 'H'),
 (1096, 'D1', 2011, '2012-02-18', 'Freiburg', 'Bayern Munich', 0, 0, 'D')]

In [258]:
match_df = pd.read_sql_query("select * from Matches WHERE Season = 2011 AND Div != 'D2'", conn)
unique_match_date = match_df['Date'].unique()

In [215]:
# alternate approach
# weather_dates = {}
# for date in unique_match_date[0:2]:
#     response = requests.get('https://api.darksky.net/forecast/'+api_key+'/52.520008,13.404954,'+date+'T12:00:00?exclude=hourly,currently,flags') 
#     if response.status_code == 200: #200 is an HTTP status code that means "everything is ok". ITs good to check the status_code in an if statement to avoid throwing and error
#         response_dict = response.json()
#         icon = response_dict['daily']['data'][0]['icon']
#         weather_dates[date] = icon
        
#     else:
#         print("Error: unable to retrieve books.  Server responded with status code", response.status_code)
    



In [216]:
weather_dates

{'2012-03-31': 'rain', '2011-12-11': 'partly-cloudy-day'}

In [259]:
weather_dates = []
for date in unique_match_date[0:5]:
    date_icon = {}
    response = requests.get('https://api.darksky.net/forecast/'+api_key+'/52.520008,13.404954,'+date+'T12:00:00?exclude=hourly,currently,flags') 
    if response.status_code == 200: #200 is an HTTP status code that means "everything is ok". ITs good to check the status_code in an if statement to avoid throwing and error
        response_dict = response.json()
        date_icon['Icon'] = response_dict['daily']['data'][0]['icon']
        date_icon['Date'] = date
        weather_dates.append(date_icon)
    else:
        print("Error: unable to retrieve books.  Server responded with status code", response.status_code)

In [271]:
df_weather = pd.DataFrame(weather_dates)
df_weather.head()

,Date,Icon
0,2012-03-31,rain
1,2011-12-11,partly-cloudy-day
2,2011-08-13,partly-cloudy-day
3,2011-11-27,partly-cloudy-day
4,2012-02-18,partly-cloudy-day


In [272]:
joined_weather_match = match_df.merge(df_weather, on='Date')

In [276]:
joined_weather_match.head(10)

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Icon
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,rain
1,1189,D1,2011,2012-03-31,Augsburg,FC Koln,2,1,H,rain
2,1206,D1,2011,2012-03-31,Leverkusen,Freiburg,0,2,A,rain
3,1215,D1,2011,2012-03-31,Kaiserslautern,Hamburg,0,1,A,rain
4,1420,D1,2011,2012-03-31,Werder Bremen,Mainz,0,3,A,rain
5,1475,D1,2011,2012-03-31,Hertha,Wolfsburg,1,4,A,rain
6,44794,E0,2011,2012-03-31,Aston Villa,Chelsea,2,4,A,rain
7,44795,E0,2011,2012-03-31,Everton,West Brom,2,0,H,rain
8,44796,E0,2011,2012-03-31,Fulham,Norwich,2,1,H,rain
9,44797,E0,2011,2012-03-31,Man City,Sunderland,3,3,D,rain


In [302]:
for team in df_team_stats_2011["TeamName"]:
    for i in joined_weather_match['HomeTeam']:
        if (i == team)and (joined_weather_match['Icon'] == 'rain'):


SyntaxError: unexpected EOF while parsing (<ipython-input-302-c961d236c9ff>, line 3)

In [304]:
joined_weather_match['Winner'].map(lambda x: joined_weather_match[] ) 


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().